# Exploring ICS ATT&CK

## Query ATT&CK

### Import TAXII Libraries
ATT&CK users can use the initial Server class to instantiate a server object pointing to the framework’s public TAXII server URL https://cti-taxii.mitre.org/taxii/

In [1]:
from taxii2client.v20 import Server

import logging
logging.getLogger('taxii2client').setLevel(logging.CRITICAL)

In [2]:
server = Server("https://cti-taxii.mitre.org/taxii/")

Available API Roots can be referenced from the server object. API Roots are logical groupings of TAXII Channels and Collections and can be thought of as instances of the TAXII API available at different URLs, where each API Root is the “root” URL of that particular instance of the TAXII API:

In [3]:
server.api_roots

In [4]:
api_root = server.api_roots[0]

### Explore ATT&CK TAXII Collections
The **collections** attribute can then be used and get more information about them via their respective available properties:

In [5]:
api_root.collections

In [6]:
for collection in api_root.collections:
    print(collection.title, "->", collection.description)

Enterprise ATT&CK -> This data collection holds STIX objects from Enterprise ATT&CK
PRE-ATT&CK -> This data collection holds STIX objects from PRE-ATT&CK
Mobile ATT&CK -> This data collection holds STIX objects from Mobile ATT&CK
ICS ATT&CK -> This data collection holds STIX objects from ICS ATT&CK


In [7]:
api_root.collections[3].title

'ICS ATT&CK'

In [8]:
api_root.collections[3].id

'02c3ef24-9cd4-48f3-a99f-b74ce24f1d34'

## Set ICS ATT&CK TAXII Collection ID Variable

In [9]:
ICS_ATTACK = "02c3ef24-9cd4-48f3-a99f-b74ce24f1d34"

## Initialize TAXII Collection Sources
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), the [TAXIICollectionSource API](https://stix2.readthedocs.io/en/latest/api/datastore/stix2.datastore.taxii.html#stix2.datastore.taxii.TAXIICollectionSource) provides an interface for searching/retrieving STIX objects from a local/remote TAXII Collection endpoint. In our case, we are pointing to our ATT&CK TAXII Collection instances (https://cti-taxii.mitre.org/stix/collections/<Collection ID>)

In [10]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

In [11]:
ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
ICS_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ICS_ATTACK + "/")
TC_ICS_SOURCE = TAXIICollectionSource(ICS_COLLECTION)

## Retrieve all ICS Techniques
Now that we can query the ICS ATT&CK TAXIICollection. We can use the query method and a set of filter to retrieve STIX objects of type "attack-pattern" -> "Techniques"

In [12]:
ICS_TECHNIQUES = TC_ICS_SOURCE.query(Filter("type", "=", "attack-pattern"))
ICS_TECHNIQUES[0]

AttackPattern(type='attack-pattern', id='attack-pattern--35392fb4-a31d-4c6a-b9f2-1c65b7f5e6b9', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2021-10-14T15:25:32.143Z', modified='2021-10-14T15:25:32.143Z', name='Transient Cyber Asset', description='Adversaries may target devices that are transient across ICS networks and external networks. Normally, transient assets are brought into an environment by authorized personnel and do not remain in that environment on a permanent basis. (Citation: NERC June 2021) Transient assets are commonly needed to support management functions and may be more common in systems where a remotely managed asset is not feasible, external connections for remote access do not exist, or 3rd party contractor/vendor access is required.\n\nAdversaries may take advantage of transient assets in different ways. For instance, adversaries may target a transient asset when it is connected to an external network and then leverage its trusted acc

In [13]:
for TECHNIQUE in ICS_TECHNIQUES:
    print(TECHNIQUE['external_references'][0]['external_id'], "--", TECHNIQUE['name'])

T0864 -- Transient Cyber Asset
T0888 -- Remote System Information Discovery
T0834 -- Native API
T0890 -- Exploitation for Privilege Escalation
T0889 -- Modify Program
T0821 -- Modify Controller Tasking
T0886 -- Remote Services
T0837 -- Loss of Protection
T0878 -- Alarm Suppression
T0806 -- Brute Force I/O
T0885 -- Commonly Used Port
T0810 -- Data Historian Compromise
T0815 -- Denial of View
T0818 -- Engineering Workstation Compromise
T0866 -- Exploitation of Remote Services
T0824 -- I/O Module Discovery
T0826 -- Loss of Availability
T0829 -- Loss of View
T0849 -- Masquerading
T0836 -- Modify Parameter
T0840 -- Network Connection Enumeration
T0844 -- Program Organization Units
T0850 -- Role Identification
T0851 -- Rootkit
T0865 -- Spearphishing Attachment
T0882 -- Theft of Operational Information
T0860 -- Wireless Compromise
T0802 -- Automated Collection
T0875 -- Change Program State
T0884 -- Connection Proxy
T0811 -- Data from Information Repositories
T0868 -- Detect Operating Mode
T08

## ICS ATT&CK Available since attackcti 0.3.4.3
Reference: https://pypi.org/project/attackcti/

In [14]:
from attackcti import attack_client
lift = attack_client()

ICS_TECHNIQUES = lift.get_ics_techniques()
print("Techniques Count:",len(ICS_TECHNIQUES))

Techniques Count: 78


In [15]:
ICS_TECHNIQUES[0]

AttackPattern(type='attack-pattern', id='attack-pattern--35392fb4-a31d-4c6a-b9f2-1c65b7f5e6b9', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2021-10-14T15:25:32.143Z', modified='2021-10-14T15:25:32.143Z', name='Transient Cyber Asset', description='Adversaries may target devices that are transient across ICS networks and external networks. Normally, transient assets are brought into an environment by authorized personnel and do not remain in that environment on a permanent basis. (Citation: NERC June 2021) Transient assets are commonly needed to support management functions and may be more common in systems where a remotely managed asset is not feasible, external connections for remote access do not exist, or 3rd party contractor/vendor access is required.\n\nAdversaries may take advantage of transient assets in different ways. For instance, adversaries may target a transient asset when it is connected to an external network and then leverage its trusted acc

## Get All Data Sources Mapped to ICS ATT&CK Techniques

In [16]:
ICS_DATA_SOURCES = []
for TECHNIQUE in ICS_TECHNIQUES:
    if 'x_mitre_data_sources' in TECHNIQUE.keys():
        for DS in TECHNIQUE['x_mitre_data_sources']:
            if DS not in ICS_DATA_SOURCES:
                ICS_DATA_SOURCES.append(DS)
ICS_DATA_SOURCES

['Network Traffic: Network Flows',
 'Network Traffic: Network Connections',
 'Assets: Asset Inventory',
 'Network Traffic: Network Traffic Content',
 'Application Log: Application Log Content',
 'Process: OS API Execution',
 'File: File Modification',
 'Asset: Software/Firmware',
 'Command: Command Execution',
 'Logon Session: Logon Session Creation',
 'Network Share: Network Share Access',
 'Network Traffic: Network Connection Creation',
 'Network Traffic: Network Traffic Flow',
 'Process: Process Creation',
 'Operational Databases: Process History/Live Data',
 'Operational Databases: Process/Event Alarm',
 'File: File Metadata',
 'Scheduled Job: Scheduled Job Metadata',
 'Scheduled Job: Scheduled Job Modification',
 'Service: Service Creation',
 'Service: Service Metadata',
 'Operational Databases: Device Alarm',
 'Asset: Device Configuration/Parameters',
 'Drive: Drive Modification',
 'Firmware: Firmware Modification',
 'Module: Module Load',
 'File: File Access',
 'Script: Script E

## Get All Groups from ICS ATT&CK

In [17]:
ICS_GROUPS = lift.get_ics_groups()
for GROUP in ICS_GROUPS:
    print(GROUP['name'])

TEMP.Veles
Dragonfly 2.0
HEXANE
APT33
OilRig
Dragonfly
Sandworm Team
Lazarus Group
ALLANITE


## Get All Malware from ICS ATT&CK

In [18]:
ICS_MALWARE = lift.get_ics_malware()
for MALWARE in ICS_MALWARE:
    print(MALWARE['name'])

Conficker
EKANS
Bad Rabbit
KillDisk
Industroyer
Stuxnet
REvil
Ryuk
LockerGoga
Triton
VPNFilter
PLC-Blaster
NotPetya
WannaCry
Flame
Backdoor.Oldrea
ACAD/Medre.A
BlackEnergy
Duqu
